In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor
%store -r conflicts
%store -r editions
%store -r all_actions
%store -r calculator
%store -r editor_inputname

editor_editions = editions[editions['editor_id']==the_editor.userid]

# Detecting conflicting editors

In [ ]:
conflicting_actions = calculator.df[calculator.conflicts.shift(-1) & (
    calculator.df.shift(-1)['editor']==editor_inputname)]

from IPython.display import display, Markdown as md
display(md('**List of conflicting editors:**'))
conf_eds = conflicting_actions['editor'].unique()
print('\n'.join(conf_eds))

display(md('**List of ids of conflicting editors:**'))
conf_ed_ids = set(int(x.split('|')[0]) for x in conf_eds)
print('\n'.join(str(x) for x in conf_ed_ids))

display(md('**List of conflicting revisions:**'))
conf_rev = conflicting_actions['rev_id'].unique()
print('\n'.join(str(x) for x in conf_rev))


# Activity of conflicting editors

In [ ]:
conf_editor_editions = editions[editions['editor_id'].isin(conf_ed_ids)]

# Visualization
from visualization.editions_listener import DFListener
from ipywidgets import interact
listener = DFListener(conf_editor_editions)
actions = (conf_editor_editions.loc[:,'actions':'actions_stopword_count'].columns.append(
    conf_editor_editions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()


interact(listener.editions_per_month, 
         begin=conf_editor_editions.year_month,
         end=conf_editor_editions.year_month.sort_values(ascending=False),
         granularity=['Monthly', 'Yearly'],
         black=actions, red=['actions_surv_48h'] + actions + ['None'],
         green=['None'] + actions, blue=['None'] + actions)

# Conflictive tokens of conflicting authors

In [ ]:
from wordcloud import WordCloud
editor_conflicts = conflicts[conflicts['editor'].isin(conf_eds)]
bow = editor_conflicts.groupby('token').size()

# Create word cloud
wc = WordCloud(width=800, height=400,background_color='white',
               max_font_size = 50,collocations=False).generate_from_frequencies(bow)


# Revisions involved
from IPython.display import display, Markdown as md
display(md(f"### The below token conflicts ocurred in a total of {len(editor_conflicts['rev_id'].unique())} revisions:"))

# Plot
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
from IPython.display import HTML
HTML('<a href="5. Using Wikiwho to explore the history of an editor.ipynb" target="_blank">Go to next workbook</a>')
